In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

In [1]:
import pandas as pd

## Load Dataset

In [32]:
train = pd.read_csv('data/train.csv', index_col = "PassengerId")

print(train.shape)
train.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
test = pd.read_csv('data/test.csv', index_col = "PassengerId")

print(test.shape)
test.head() 

(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## 잘하는 프로그래머 데이터사이언티스트와 못하는 프로그래머 데이터사이언티스트의 차이는 Tool의 차이가 아니라 습관의 차이다.
## 실제론 에러가 나지 않았더라도 잘 동작하지 않은 경우가 있음
## 이런 경우 나중에 오히려 더 큰 문제가 생김
## 그러므로 좋은 프로그래머 데이터사이언티스트는 코드가 잘 동작했다고 해서
## 100% 맹신하지 않음
## 언제나 결과를 출력하고 눈으로 확인하는 '습관'이 있음

## Preprocessing

### 전처리과정

### 1. Text로 이루어진 컬럼(성별이나 Embarked와 같은)을 따로 처리하는 과정
### 2. 비어있는 값을 채움

### Encode Sex

In [34]:
train['Sex_encode'] = train['Sex'].replace('male', 0).replace('female', 1)

print(train.shape)
train[['Sex', 'Sex_encode']].head()

(891, 12)


,Sex,Sex_encode
PassengerId,,
1,male,0
2,female,1
3,female,1
4,female,1
5,male,0


In [35]:
test['Sex_encode'] = test['Sex'].replace('male', 0).replace('female', 1)

print(test.shape)
test[['Sex', 'Sex_encode']].head()

(418, 11)


,Sex,Sex_encode
PassengerId,,
892,male,0
893,female,1
894,male,0
895,male,0
896,female,1


### Fill in missing Fare

In [36]:
# train 데이터에는 빈값이 없음
# test 데이터에는 빈값이 1개 있음
# 보통은 train 데이터에서의 평균을 구해 넣지만
# 지금 같은 경우 비어있는 값이 1개밖에 없으므로 그냥 0으로 때움

test['Fare'].fillna(0) # 이렇게만 하면 안고쳐짐, 원본을 훼손하는 것에 거부감이 있으므로

test['Fare'] = test['Fare'].fillna(0)

test[test['Fare'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode
PassengerId,,,,,,,,,,,


### Encode Embarked

In [63]:
# C == 0, S == 1, Q == 2 이렇게 인코딩하면 Decision Tree가 오해할 수 있음
# 2 * S == Q?
# S + S == Q?
# 성별처럼 2개인 경우는 0, 1로 바꾸면 되지만
# 경우의 수가 3가지가 넘어가는 경우

# One Hot Encoding으로 바꿔줘야 함
# C == [1, 0, 0]
# S == [0, 1, 0]
# Q == [0, 0, 1]

# 파이썬 기초 상식: True == 1, False == 0
# C == [True, False, False]
# S == [False, True, False]
# Q == [True, True, True]
 

train['Embarked_C'] = train['Embarked'] == 'C'
train['Embarked_S'] = train['Embarked'] == 'S'
train['Embarked_Q'] = train['Embarked'] == 'Q'

print(train.shape)
train[['Embarked', 'Embarked_C', 'Embarked_S', 'Embarked_Q']]

(891, 15)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
1,S,False,True,False
2,C,True,False,False
3,S,False,True,False
4,S,False,True,False
5,S,False,True,False
...,...,...,...,...
887,S,False,True,False
888,S,False,True,False
889,S,False,True,False


In [64]:
test['Embarked_C'] = test['Embarked'] == 'C'
test['Embarked_S'] = test['Embarked'] == 'S'
test['Embarked_Q'] = test['Embarked'] == 'Q'

print(test.shape)
test[['Embarked', 'Embarked_C', 'Embarked_S', 'Embarked_Q']]

(418, 14)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
892,Q,False,False,True
893,S,False,True,False
894,Q,False,False,True
895,S,False,True,False
896,S,False,True,False
...,...,...,...,...
1305,S,False,True,False
1306,C,True,False,False
1307,S,False,True,False


In [66]:
feature_names = ['Pclass', 'Sex_encode', 'Fare', 'Embarked_C', 'Embarked_S', 'Embarked_Q']
feature_names

['Pclass', 'Sex_encode', 'Fare', 'Embarked_C', 'Embarked_S', 'Embarked_Q']

## Train

In [37]:
# Feature - Pclass, Sex, Fare, ...
# Label - Survived

In [38]:
feature_names = ['Pclass', 'Sex_encode', 'Fare']
feature_names

['Pclass', 'Sex_encode', 'Fare']

In [39]:
X_train = train[feature_names]

print(X_train.shape)
X_train.head()

(891, 3)


,Pclass,Sex_encode,Fare
PassengerId,,,
1,3,0,7.2500
2,1,1,71.2833
3,3,1,7.9250
4,1,1,53.1000
5,3,0,8.0500


In [40]:
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

(418, 3)


,Pclass,Sex_encode,Fare
PassengerId,,,
892,3,0,7.8292
893,3,1,7.0000
894,2,0,9.6875
895,3,0,8.6625
896,3,1,12.2875


In [42]:
label_name = 'Survived'

y_train = train[label_name]

print(y_train.shape)
y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

### Use Decision Tree

In [44]:
# scikit-learn = sklearn
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 5) # Fare는 가지를 너무 많이 치므로 max_depth걸어둠
model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [48]:
# 1) fit(train) -> 2) predict(test)
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [49]:
prediction = model.predict(X_test)

print(prediction.shape)
prediction[0:5]

(418,)


array([0, 1, 0, 0, 0], dtype=int64)

### Submit

In [54]:
submit = pd.read_csv('data/gender_submission.csv', index_col = 'PassengerId')

submit['Survived'] = prediction

print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,0


In [55]:
submit.to_csv('data/Decision-tree.csv')